In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms
from PIL import Image
import os
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm

from models import *
from function import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


C:\Users\DSP523\AppData\Local\Temp\ipykernel_5388\3601183918.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
# Argument
batch_size = 32
epochs = 10

In [3]:
train_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [4]:
train_set = trainDataset('retina-train', 'train_labels.csv', train_tfm)
train_set_size = int(len(train_set) * 0.8)
train_set, valid_set = torch.utils.data.random_split(train_set, [train_set_size, len(train_set) - train_set_size])

test_set = testDataset('retina-test', train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=0)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

In [13]:
model = Classification(3, 3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)
softmax = nn.Softmax(dim = -1)

for epoch in range(epochs):
    train_loss = 0
    train_accs = 0
    total = 0
    model.train()
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
     
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        train_accs += (preds == labels).sum().item()
    scheduler.step()
    print(f'Epoch {epoch+1} / {epochs} | train_accs = {train_accs / total} & train_loss = {train_loss / total}')

    model.eval()
    valid_loss = 0.0
    valid_accs = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            imgs, labels = batch
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            _, preds= torch.max(outputs, 1)
            total += labels.size(0)
            valid_accs += (preds == labels).sum().item()

    print(f'Epoch {epoch+1} / {epochs} | validation_accs = {valid_accs / total} & validation_loss = {valid_loss / total}')

torch.save(model.state_dict(), './checkpoints/proj1_baseline.pth')

  0%|          | 0/53 [00:00<?, ?it/s]

100%|██████████| 53/53 [00:09<00:00,  5.55it/s]


Epoch 0 / 10 | train_accs = 0.579136690647482 & train_loss = 0.030703417688822576


100%|██████████| 14/14 [00:01<00:00, 11.97it/s]


Epoch 1 / 10 | validation_accs = 0.5956937799043063 & validation_loss = 0.03213127092881636


...

Epoch 9 / 10 | validation_accs = 0.5956937799043063 & validation_loss = 0.032092834488626876


100%|██████████| 53/53 [00:09<00:00,  5.51it/s]


Epoch 9 / 10 | train_accs = 0.579136690647482 & train_loss = 0.03054142916659943


100%|██████████| 14/14 [00:01<00:00, 11.57it/s]


Epoch 10 / 10 | validation_accs = 0.5956937799043063 & validation_loss = 0.032093241026527004


In [14]:
preds_list = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        imgs = batch
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, preds= torch.max(outputs, 1)
        preds_list += preds.tolist()

name = sorted(os.listdir("retina-test"))
for i in range(len(name)):
    name[i], _ = name[i].split('.')

submission ={"image" : name, "level" : preds_list}
file = pd.DataFrame(submission)
file.to_csv("submission1.csv", index = False)

100%|██████████| 17/17 [00:01<00:00, 11.70it/s]
